In [192]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go # for visualization
import math
# making data frame  
df = pd.read_excel("Hot_and_Cold_final.xlsx")

In [193]:
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month

In [194]:
#Hot Water
new_df = df.loc[df['Water type'] == 1]
#Cold Water
#new_df = df.loc[df['Water type'] == 0]

In [195]:
new_df = new_df[new_df['Month'].between(6, 8)]

In [196]:
#Choosing the important features in order to make a correlation
#Hot Water
new_df = new_df[['Chlorine mg\l','Turbidity NTU','Water Temperature','Day Before Min','HPC CFU/ml']]
#Cold Water
#new_df = new_df[['Chlorine mg\l','Turbidity NTU','Day Before Min','HPC CFU/ml']]

In [197]:
new_df.drop(new_df.index[new_df['Chlorine mg\l'] == 'ND'], inplace = True)
new_df.drop(new_df.index[new_df['Turbidity NTU'] == 'ND'], inplace = True)
new_df = new_df.astype(float)

In [198]:
new_df = new_df.dropna()
len(new_df)

929

In [199]:
new_df.loc[(new_df['HPC CFU/ml'] == 0),'HPC CFU/ml'] = 0.001

In [200]:
new_df['HPC CFU/ml'] = np.log(new_df['HPC CFU/ml'])

In [201]:
corr_df = new_df.corr(method='spearman')

fig = go.Figure()
fig.add_traces(go.Heatmap(
    z=corr_df, 
    x=corr_df.columns, 
    y=corr_df.columns,
    zmax=1, 
    zmin=-1
))
fig.update_layout({
    'title':"Features Correlation Heatmap"
})

In [202]:
corr_df

,Chlorine mg\l,Turbidity NTU,Water Temperature,Day Before Min,HPC CFU/ml
Chlorine mg\l,1.000000,-0.164899,-0.031751,-0.174141,-0.245114
Turbidity NTU,-0.164899,1.000000,-0.135295,0.156935,0.067869
Water Temperature,-0.031751,-0.135295,1.000000,-0.005217,-0.243013
Day Before Min,-0.174141,0.156935,-0.005217,1.000000,0.094022
HPC CFU/ml,-0.245114,0.067869,-0.243013,0.094022,1.000000


In [203]:
y = new_df['HPC CFU/ml']

#Hot Water
X = new_df[['Chlorine mg\l','Turbidity NTU','Water Temperature','Day Before Min']]
#Cold Water
#X = new_df[['Chlorine mg\l','Turbidity NTU','Day Before Min']]
X

,Chlorine mg\l,Turbidity NTU,Water Temperature,Day Before Min
5846,0.00,0.52,45.0,23.3
5864,0.00,0.35,60.0,18.9
5865,0.00,0.85,54.0,18.9
5871,0.00,0.40,44.0,17.1
5872,0.00,0.46,44.0,17.1
...,...,...,...,...
26901,0.11,1.54,63.0,21.5
26902,0.21,3.61,61.0,21.5
26903,0.32,2.11,61.0,21.5
26995,0.00,0.88,60.0,26.0


In [204]:
#split the data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1357)
train_df = pd.merge(left=X_train, right=y_train, left_index=True, right_index=True)
test_df = pd.merge(left=X_test, right=y_test, left_index=True, right_index=True)
test_df.head(10)

,Chlorine mg\l,Turbidity NTU,Water Temperature,Day Before Min,HPC CFU/ml
15732,0.21,0.45,48.6,21.1,4.962845
23685,0.20,0.85,53.0,23.8,1.386294
18606,0.00,0.75,53.0,25.3,3.663562
8550,0.00,0.68,53.0,22.2,0.000000
8260,0.00,0.40,42.0,22.8,0.000000
18269,0.02,0.49,45.0,20.9,4.553877
23671,0.20,0.36,55.6,18.1,4.875197
26782,0.00,0.73,55.2,22.2,0.693147
23922,0.20,0.86,28.0,23.7,1.098612
21454,0.40,0.70,56.0,26.0,-6.907755


In [205]:
#scale the sets
from sklearn.preprocessing import StandardScaler

x_train_scaler = StandardScaler()
x_test_scaler = StandardScaler()
y_train_scaler = StandardScaler()
y_test_scaler = StandardScaler()

X_train_scaled = x_train_scaler.fit_transform(X_train)
X_test_scaled = x_test_scaler.fit_transform(X_test)
y_train_scaled = y_train_scaler.fit_transform(pd.DataFrame(y_train))
y_test_scaled = y_test_scaler.fit_transform(pd.DataFrame(y_test))

In [206]:
#Hot Water
correlated = ['Chlorine mg\l','Turbidity NTU','Water Temperature','Day Before Min']
#Cold Water
#correlated = ['Chlorine mg\l','Turbidity NTU','Day Before Min']

In [207]:
from sklearn.linear_model import LinearRegression
lm = LinearRegression()  # define our model using least square method
lm.fit(X_train_scaled,y_train_scaled)              # Fit our linear model
# The coefficients
coeff = lm.coef_[0]
intercept = lm.intercept_[0]
coeffstr = ""
for i in range(len(correlated)):
  coeffstr+=" + "+str(round(coeff[i], 3))
  coeffstr+=" * "+str(correlated[i])

print('Coefficients: \n', "coeff =", coeff , ",  Intercept=",intercept,"\n" )
print("The regression equation is:" ,"HPC = " ,intercept,"",coeffstr)

Coefficients: 
 coeff = [-0.19031924 -0.02129291 -0.15336197  0.13355506] ,  Intercept= 3.1153695896930997e-16 

The regression equation is: HPC =  3.1153695896930997e-16   + -0.19 * Chlorine mg\l + -0.021 * Turbidity NTU + -0.153 * Water Temperature + 0.134 * Day Before Min


In [208]:
fitted_scaled_HPC_train = lm.predict(X_train_scaled) # returns ndarray
#inverse the minmax scaler transformation
fitted_HPC_train = y_train_scaler.inverse_transform(fitted_scaled_HPC_train)

In [209]:
#create a pandas series of the results
predicted_train_HPC = round(pd.Series(fitted_HPC_train[:,0], index=y_train.index, name='Predicted_train_HPC'),ndigits=2)

In [210]:
#Add the results to the DF
train_df = pd.merge(left=train_df, right=predicted_train_HPC , left_index=True, right_index=True)

In [211]:
fitted_scaled_HPC_test = lm.predict(X_test_scaled)
fitted_HPC_test = y_test_scaler.inverse_transform(fitted_scaled_HPC_test)
predicted_test_HPC = round(pd.Series(fitted_HPC_test[:,0], index=y_test.index, name='Predicted_test_HPC'),ndigits=2)
test_df = pd.merge(left=test_df, right=predicted_test_HPC, left_index=True, right_index=True)
test_df.head(20)

,Chlorine mg\l,Turbidity NTU,Water Temperature,Day Before Min,HPC CFU/ml,Predicted_test_HPC
15732,0.21,0.45,48.6,21.1,4.962845,0.80
23685,0.20,0.85,53.0,23.8,1.386294,1.04
18606,0.00,0.75,53.0,25.3,3.663562,2.29
8550,0.00,0.68,53.0,22.2,0.000000,1.71
8260,0.00,0.40,42.0,22.8,0.000000,2.56
18269,0.02,0.49,45.0,20.9,4.553877,1.90
23671,0.20,0.36,55.6,18.1,4.875197,-0.17
26782,0.00,0.73,55.2,22.2,0.693147,1.56
23922,0.20,0.86,28.0,23.7,1.098612,2.63
21454,0.40,0.70,56.0,26.0,-6.907755,0.32


In [212]:
from sklearn import metrics
print("------ TRAIN DATA ------")
print("MSE:",metrics.mean_squared_error(train_df['HPC CFU/ml'], train_df['Predicted_train_HPC']))
print("RMSE:",np.sqrt(metrics.mean_squared_error(train_df['HPC CFU/ml'], train_df['Predicted_train_HPC'])))
print("MAE:",metrics.mean_absolute_error(train_df['HPC CFU/ml'], train_df['Predicted_train_HPC']))

------ TRAIN DATA ------
MSE: 11.337641559990663
RMSE: 3.367141452328765
MAE: 2.3151255820426


In [213]:
#get R2 for train set
from sklearn.metrics import r2_score
r2_score(train_df['HPC CFU/ml'], train_df['Predicted_train_HPC'])

0.08435960601002279

In [214]:
train_df['HPC CFU/ml'].std()

3.5215448140286796

In [215]:
print("------ TEST DATA ------")
print("MSE:",metrics.mean_squared_error(test_df['HPC CFU/ml'], test_df['Predicted_test_HPC']))
print("RMSE:",np.sqrt(metrics.mean_squared_error(test_df['HPC CFU/ml'], test_df['Predicted_test_HPC'])))
print("MAE:",metrics.mean_absolute_error(test_df['HPC CFU/ml'], test_df['Predicted_test_HPC']))

------ TEST DATA ------
MSE: 12.988110649367915
RMSE: 3.603902142035479
MAE: 2.596418953302923


In [216]:
#get R2 for test set
r2_score(test_df['HPC CFU/ml'], test_df['Predicted_test_HPC'])

0.01006099000793248

In [217]:
test_df['HPC CFU/ml'].std()

3.6286783561103086